# **Mini-Formation : Utilisation des API dans les Pipelines de Données**
Ce notebook vous guidera à travers l'utilisation des **API REST** pour l'ingestion de données dans un pipeline de données.

**Objectifs :**
- Comprendre les bases des API REST (GET, POST, PUT, DELETE)
- Extraire des données d'APIs publiques et les stocker dans une base de données
- Automatiser l'ingestion de données via un pipeline
- Gérer l'authentification et la pagination


## **1. Installation des Bibliothèques Nécessaires**

In [17]:
%pip install requests pandas sqlalchemy

Note: you may need to restart the kernel to use updated packages.


## **2. Importation des Modules**

In [18]:

import requests
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import json


## **3. Introduction aux API REST**
Une API REST (Representational State Transfer) permet d'échanger des données entre des systèmes via des requêtes HTTP. Les méthodes principales sont :

- `GET` : Récupérer des données
- `POST` : Envoyer de nouvelles données
- `PUT` : Mettre à jour des données existantes
- `DELETE` : Supprimer des données

Nous allons maintenant faire des appels API et manipuler les données reçues.

## **4. Effectuer une Requête API avec `requests`**

In [19]:

# URL d'une API publique (exemple : JSONPlaceholder)
url = "https://jsonplaceholder.typicode.com/posts"

# Faire une requête GET
response = requests.get(url)

# Vérifier si la requête a réussi
if response.status_code == 200:
    data = response.json()
    print("Données récupérées avec succès !")
    print(json.dumps(data[:2], indent=2))  # Affiche les 2 premiers résultats
else:
    print("Erreur lors de la requête :", response.status_code)


Données récupérées avec succès !
[
  {
    "userId": 1,
    "id": 1,
    "title": "sunt aut facere repellat provident occaecati excepturi optio reprehenderit",
    "body": "quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto"
  },
  {
    "userId": 1,
    "id": 2,
    "title": "qui est esse",
    "body": "est rerum tempore vitae\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\nfugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis\nqui aperiam non debitis possimus qui neque nisi nulla"
  }
]


## **5. Transformer les Données API en DataFrame**

In [20]:

# Convertir les données JSON en DataFrame pandas
df = pd.DataFrame(data)
print(df.head())  # Afficher les premières lignes


   userId  id                                              title  \
0       1   1  sunt aut facere repellat provident occaecati e...   
1       1   2                                       qui est esse   
2       1   3  ea molestias quasi exercitationem repellat qui...   
3       1   4                               eum et est occaecati   
4       1   5                                 nesciunt quas odio   

                                                body  
0  quia et suscipit\nsuscipit recusandae consequu...  
1  est rerum tempore vitae\nsequi sint nihil repr...  
2  et iusto sed quo iure\nvoluptatem occaecati om...  
3  ullam et saepe reiciendis voluptatem adipisci\...  
4  repudiandae veniam quaerat sunt sed\nalias aut...  


## **6. Stockage des Données API dans une Base de Données**

In [21]:
import sqlite3
import pandas as pd

# Connexion directe à SQLite
conn = sqlite3.connect("api_data.db")

# Stocker le DataFrame en SQLite
df.to_sql("posts", conn, if_exists="replace", index=False)

print("Données enregistrées avec succès dans SQLite.")

# Vérifier les données
df_from_db = pd.read_sql("SELECT * FROM posts", conn)
print(df_from_db)

# Fermer la connexion
conn.close()


Données enregistrées avec succès dans SQLite.
    userId   id                                              title  \
0        1    1  sunt aut facere repellat provident occaecati e...   
1        1    2                                       qui est esse   
2        1    3  ea molestias quasi exercitationem repellat qui...   
3        1    4                               eum et est occaecati   
4        1    5                                 nesciunt quas odio   
..     ...  ...                                                ...   
95      10   96  quaerat velit veniam amet cupiditate aut numqu...   
96      10   97         quas fugiat ut perspiciatis vero provident   
97      10   98                        laboriosam dolor voluptates   
98      10   99  temporibus sit alias delectus eligendi possimu...   
99      10  100              at nam consequatur ea labore ea harum   

                                                 body  
0   quia et suscipit\nsuscipit recusandae consequu...  
1

## **7. Automatisation du Pipeline API**
Nous allons créer une fonction qui :
- Fait une requête API
- Transforme les données
- Stocke les résultats en base de données
- Peut être exécutée automatiquement


In [23]:
import requests
import pandas as pd
import sqlite3

# Connexion à SQLite avec sqlite3
conn = sqlite3.connect("api_data.db")  # Crée ou ouvre la base SQLite

def fetch_and_store_data(api_url, table_name, conn):
    """Récupère des données depuis une API et les stocke dans une base SQLite"""
    response = requests.get(api_url)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)

        # Stocker directement avec sqlite3
        df.to_sql(table_name, conn, if_exists="replace", index=False)

        print(f"Données mises à jour dans la table {table_name}.")
    else:
        print(f"Échec de la récupération des données. Code : {response.status_code}")

# Exécuter le pipeline
fetch_and_store_data("https://jsonplaceholder.typicode.com/comments", "comments", conn)

# Vérifier les données enregistrées
df_from_db = pd.read_sql("SELECT * FROM comments", conn)
print(df_from_db.head())

# Fermer la connexion
conn.close()


Données mises à jour dans la table comments.
   postId  id                                       name  \
0       1   1               id labore ex et quam laborum   
1       1   2  quo vero reiciendis velit similique earum   
2       1   3              odio adipisci rerum aut animi   
3       1   4                             alias odio sit   
4       1   5      vero eaque aliquid doloribus et culpa   

                    email                                               body  
0      Eliseo@gardner.biz  laudantium enim quasi est quidem magnam volupt...  
1  Jayne_Kuhic@sydney.com  est natus enim nihil est dolore omnis voluptat...  
2     Nikita@garfield.biz  quia molestiae reprehenderit quasi aspernatur\...  
3           Lew@alysha.tv  non et atque\noccaecati deserunt quas accusant...  
4       Hayden@althea.biz  harum non quasi et ratione\ntempore iure ex vo...  


## **8. Conclusion**
- Nous avons vu comment **extraire des données depuis une API**.
- Nous avons appris à **stocker ces données en base de données**.
- Nous avons automatisé un **pipeline de récupération des données**.

**À tester :** Essayez d'exécuter ce pipeline avec une autre API publique ! 🚀